### Lesson 6 


#### Random Forest

[How random forests work](https://www.kaggle.com/code/jhoward/how-random-forests-really-work/)

In [1]:
import os
from pathlib import Path
from fastai.imports import *
path = Path('data/titanic')
df = pd.read_csv(path/'train.csv')
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
from fastai.imports import * # imports a ton of crap

def proc_data(df):
    df['Fare'] = df.Fare.fillna(0)
    df.fillna(modes, inplace=True)
    df['LogFare'] = np.log1p(df['Fare'])
    df['Embarked'] = pd.Categorical(df.Embarked)
    df['Sex'] = pd.Categorical(df.Sex)

proc_data(df)

# I am not bothering with the test data for now, i dont plan to submit this.

Note that he uses pd.Categorical to use pandas direct support for categorical.  Note that Pclass is ordered, so we leave it be as decision trees can handle these just fine as is.  We wont need to do dummy variables at all here.

In [ ]:
cats=["Sex","Embarked"]
conts=['Age', 'SibSp', 'Parch', 'LogFare',"Pclass"]
dep="Survived"

## More highlights Chapter 9

*  If OOB significantly better then validation error, consider that something beyond generalization error is causing underfitting.

* Interpretation
    * Standard deviation of the trees results give a measure of confidence on the trees prediction
    * SKlearn can provide feature importance (computed by looking at how much the model improved when splitting on those features).
    * Feature importance can be used to select unimportant features to remove and simplify the model.
    * `cluster_columns`  can be used to identify redundant columns to try and remove (checking `oob` score to see if it is effected ).  `cluster_columns` is defined in the books `utils.py`  as:

        ```
        from scipy.cluster import hierarchy as hc

        def cluster_columns(df, figsize=(10,6), font_size=12):
            corr = np.round(scipy.stats.spearmanr(df).correlation, 4)
            corr_condensed = hc.distance.squareform(1-corr)
            z = hc.linkage(corr_condensed, method='average')
            fig = plt.figure(figsize=figsize)
            hc.dendrogram(z, labels=df.columns, orientation='left', leaf_font_size=font_size)
            plt.show()
        ```
    * Partial dependance plots help understand how the results depend on each variable, all other variables held constant. (Counterfactual!)
    * Library `treeinterpreter` can be used to show for predicting a particular row of data, what were the most important factors, and how did they influence the prediction.  


* Data Leakage: Be careful, sometime data is missing for a reason, and that reason might be related to the value you are trying to predict!!  Models can help... are some variables more predictive the you expect? Do some variables partial dependance not make sense?


* Random forests and related methods are bad at extrapolation (just predict constant values outside the domain fo the data)
   * Book chapter shows a cool trick to detect issues like this: Train a random forest to predict if a data point is in the test set or validation set.  
   * This can uncover "domain_shift" issues, for example in the chapter, machine_id and sales_id both encode the date of sale (albeit imperfectly) and so predict well (for this case) train vs validation.

* Fast.ai's `TabularLearner` was demonstrated next. This is by default a two hidden layer neural network that uses embedding for the categorical variables. It does slightly better.  Even better is ensembling this with the forest results! Note: Default is hidden layers are 100,200 and default embedding is `min(600, 1.6*n_cat**0.56)`

* Boosting is mentioned breifly, and that XGBoost is the state of the art at the time of writing. ALso that boosting tends to be more susceptable to overfitting and can be more sensitive to hyperparameters.

* Embeddings obtained by training a neural net (perhaps on another task!) can be used as input features for your Random forest or other machine learning methods. (Guo and Berkhahn 2016, Entity Embeddings of Categorical Variables)